# Sheffield Tree Survey

In [42]:
import csv

inp_fn = "cleanup.csv"

with open(inp_fn) as inp:
    raw_rows = list(csv.reader(inp))

Column indexes 10 and 11 are junk.

In [43]:
raw_rows = [r[:10] for r in raw_rows]

In [44]:
if 'Totals' in raw_rows[-1]:
    del raw_rows[-1]

In [45]:
header, *raw_rows = raw_rows
list(enumerate(header))

[(0, 'Title'),
 (1, 'Zone'),
 (2, 'Responses'),
 (3, 'Households'),
 (4, 'Agree'),
 (5, 'Disagree'),
 (6, 'Response\nrate'),
 (7, 'Disagree -\nrespondents (%)'),
 (8, 'Tree Panel\nRequired?'),
 (9, 'Phase')]

## Bogus counts

At least one row has bogus counts: the `agrees` + `disagrees` exceeds the numbers of responses.

In [46]:
[r for r in raw_rows if int(r[4])+int(r[5]) > int(r[2])]

[['Bannerdale Road',
  'B54 Carterknowle',
  '32',
  '272',
  '32',
  '13',
  '12%',
  '41%',
  'No',
  '2']]

## Bogus household counts

A few rows have more responses than there are households.

In [47]:
[r for r in raw_rows if int(r[2]) > int(r[3])]

[['Lyndhurst Road',
  'B54 Carterknowle',
  '2',
  '0',
  '2',
  '0',
  '9%',
  '0%',
  'No',
  '2'],
 ['Manchester Road',
  'A21 Manchester Road',
  '14',
  '1',
  '12',
  '2',
  '10%',
  '14%',
  'No',
  '2'],
 ['Montrose Road',
  'B54 Carterknowle',
  '3',
  '1',
  '3',
  '0',
  '5%',
  '0%',
  'No',
  '2'],
 ['Mortimer Road',
  'A19 Bradfield',
  '1',
  '0',
  '1',
  '0',
  '10%',
  '0%',
  'No',
  '2'],
 ['Owler Gate',
  'B10 Oughtibridge',
  '2',
  '0',
  '2',
  '0',
  '20%',
  '0%',
  'No',
  '2'],
 ['Spurr Street', 'B63 Lowfield', '1', '0', '1', '0', '3%', '0%', 'No', '2'],
 ['Willington Road',
  'B21 Firth Park',
  '1',
  '0',
  '1',
  '0',
  '3%',
  '0%',
  'No',
  '2']]

All of these seem funny.
Mortimer Road for example is perhaps one of Sheffield's longest roads.
It is a winding country lane, but surely it has _some_ households on?
Well, one household managed to respond from this road with seemingly no households on, so yes,
it must have some.
But the percentage of respondents is surely bogus?
There is one response and 0 households, and somehow the fraction of responses is marked as 10%.

Manchester Road, to pick another example, has hundreds of households on.
So what gives?

Sometimes, the `disagree` percentage is just utterly bogus.

In [48]:
def cvt(x):
    if x == '-':
        return 0
    return int(x.replace('%', ''))

In [49]:
[r for r in raw_rows if cvt(r[7]) > 100]

[['Thompson Road',
  'B48 Endcliffe',
  '4',
  '57',
  '1',
  '3',
  '7%',
  '7500%',
  'Yes',
  '5']]

The above two issues suggest that data has been transcribed by hand.
How else does `0.75` become `75` (that is, `7500%`)?
And how does Mortimer Road come to have 0 households instead of 10?

The fact that the data has been touched by human hands basically casts suspicion over it all.

Note that many cases of Independent Tree Panel referral were missed by a single `disagree` response:

In [50]:
close_cases = [r for r in raw_rows if cvt(r[2]) and cvt(r[5])/cvt(r[2]) < 0.5 and (cvt(r[5])+1)/cvt(r[2]) >= 0.5]

Here are a few of the close cases.

In [51]:
sorted(close_cases, key=lambda r: r[2])[-3:]

[['Northcote Avenue',
  'B61 - Norton Woodseats',
  '8',
  '54',
  '5',
  '3',
  '15%',
  '38%',
  'No',
  '10'],
 ['Cavendish Avenue',
  'B52 - Dore',
  '9',
  '36',
  '5',
  '4',
  '25%',
  '44%',
  'No',
  '10'],
 ['Gleadless Road',
  'A23 - Gleadless',
  '9',
  '361',
  '5',
  '4',
  '2%',
  '44%',
  'No',
  '10']]

How many close cases?

In [52]:
len(close_cases)

71

Also worth noting that because an Independent Tree Panel referral is required when `disagrees` >= 0.5 `responses`, the responses which were neither agree nor disagree are, practically speaking, the same as `agree`.

In [53]:
raw_counts = [(cvt(r[2]), cvt(r[4])+cvt(r[5])) for r in raw_rows]
sum([r-c for r,c in raw_counts if r > c])

42